In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import joblib
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report

# Load datasets
languages = ['eng_tamil']
label = {
    '0': 'homophobic',
    '1': 'normal',
    '2': 'transphobic'
}
data = {}

for lang in languages:
    data[lang] = {
        'train': pd.read_csv(f"/content/eng_tam_train_prepro_nonaug.csv"),
        'test': pd.read_csv(f"/content/eng_tam_test_prepro_nonaug.csv"),
        'dev': pd.read_csv(f"/content/eng_tam_dev_prepro_nonaug.csv")
    }

# Encode labels
label_encoder = LabelEncoder()
for lang in languages:
    data[lang]['train']['label'] = label_encoder.fit_transform(data[lang]['train']['label'])
    data[lang]['test']['label'] = label_encoder.transform(data[lang]['test']['label'])

# Feature extraction
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(data[lang]['train']['text'])
X_test = vectorizer.transform(data[lang]['test']['text'])
y_train = data[lang]['train']['label']
y_test = data[lang]['test']['label']

# Build the ANN model with increased hidden layers and dropout
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(lr=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train.toarray(), y_train, epochs=20, batch_size=32, validation_split=0.1)

# Evaluate the model
y_pred = np.argmax(model.predict(X_test.toarray()), axis=-1)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Epoch 1/20
109/109 [==============================] - 8s 44ms/step - loss: 0.4822 - accuracy: 0.8846 - val_loss: 0.3305 - val_accuracy: 0.8966
Epoch 2/20
109/109 [==============================] - 4s 39ms/step - loss: 0.2919 - accuracy: 0.8898 - val_loss: 0.3195 - val_accuracy: 0.8966
Epoch 3/20
109/109 [==============================] - 5s 51ms/step - loss: 0.2029 - accuracy: 0.8949 - val_loss: 0.3921 - val_accuracy: 0.8941
Epoch 4/20
109/109 [==============================] - 4s 34ms/step - loss: 0.1554 - accuracy: 0.9318 - val_loss: 0.4349 - val_accuracy: 0.8786
Epoch 5/20
109/109 [==============================] - 4s 34ms/step - loss: 0.1231 - accuracy: 0.9525 - val_loss: 0.5819 - val_accuracy: 0.8579
Epoch 6/20
109/109 [==============================] - 5s 42ms/step - loss: 0.1048 - accuracy: 0.9585 - val_loss: 0.7211 - val_accuracy: 0.8630
Epoch 7/20
109/109 [==============================] - 4s 34ms/step - loss: 0.0899 - accuracy: 0.9580 - val_loss: 0.7098 - val_accuracy: 0.8165